<a href="https://colab.research.google.com/github/fefranco97/Compiladores/blob/main/Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto de compiladoes**
**Alunos**
* Felipe Franco de Camargo      TIA:41541014
* Willian Testsuya Takashi      TIA:41538579

**Gramatica implementada**
* https://graduacao.mackenzie.br/pluginfile.php/508915/mod_resource/content/1/MINIPASCAL.pdf

In [ ]:
!pip install rply

##Analisador Lexico


In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('INT', r'int')
lg.add('CHAR', r'char')
lg.add('BREAK', r'break')
lg.add('CONTINUE', r'continue')
lg.add('RETURN', r'return')
lg.add('READINT', r'readint')
lg.add('WRITEINT', r'writeint')
lg.add('IF', r'if')
lg.add('WHILE', r'while')
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('OPEN_PAR', r'\(')
lg.add('CLOSE_PAR', r'\)')
lg.add('OPEN_COL', r'\[')
lg.add('CLOSE_COL', r'\]')
lg.add('VIRG', r'\,')
lg.add('OPEN_CH', r'\{')
lg.add('CLOSE_CH', r'\}')
lg.add('PVIRG', r'\;')
lg.add('COMPEQUALS', r'==')
lg.add('COMPMAIOR', r'\>')
lg.add('COMPMENOR', r'\<')
lg.add('EQUALS', r'=')
lg.add('INTERROG', r'\?')
lg.add('DOISP', r'\:')
lg.add('DISJ', r'\|\|')
lg.add('CONJ', r'&&')
lg.add('NOT', r'\!')
lg.add('NUMBER', r'\d+')
lg.add('CHARCONST', r'\'\S\'')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MOD', r'\%')
lg.add('POWER', r'\^')

lg.ignore('\s+')

lexer = lg.build()

##Analisador Sintático

In [ ]:
from rply.token import BaseBox

class Number(BaseBox):
    def __init__(self, value):
        self.value = value

    def eval(self):
        return self.value

class BinaryOp(BaseBox):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def print(self):
        return print("Valor Left: ", self.left.eval(), "\nValor Right:  ", self.right.eval())

class Add(BinaryOp):
    def eval(self):
        return self.left.eval() + self.right.eval()
    def print(self):
        return print("Valor Inicial: ", self.left.eval(), "\nValor que irá ser somado: ", self.right.eval(), "\nValor após soma: ", self.left.eval() + self.right.eval())       

class Sub(BinaryOp):
    def eval(self):
        return self.left.eval() - self.right.eval()
    def print(self):
        return print("Valor Inicial: ", self.left.eval(), "\nValor que irá ser subtraido:  ", self.right.eval(),"\nValor após subtração: ", self.left.eval() - self.right.eval())

class Mul(BinaryOp):
    def eval(self):
        return self.left.eval() * self.right.eval()
    def print(self):
        return print("Valor Inicial: ", self.left.eval(),"\nValor multiplicador:  ", self.right.eval(),"\nValor após multiplicação: ", self.left.eval() * self.right.eval())

class Div(BinaryOp):
    def eval(self):
        return self.left.eval() / self.right.eval()
    def print(self):
        return print("Valor Inicial: ", self.left.eval(),"\nValor divisor:  ", self.right.eval(), "\nValor após divisão: ", self.left.eval() / self.right.eval())

class Mod(BinaryOp):
    def eval(self):
        return self.left.eval() % self.right.eval()
    def print(self):
        return print("Valor Inicial: ", self.left.eval(), "\nValor que irá ser subtraido:  ", self.right.eval(), "\nValor após subtração: ", self.left.eval() - self.right.eval())

class CompEqual(BinaryOp):
    def eval(self):
        return self.left.eval() == self.right.eval()

class CompMenor(BinaryOp):
    def eval(self):
        return self.left.eval() < self.right.eval()

class CompMaior(BinaryOp):
    def eval(self):
        return self.left.eval() > self.right.eval()

class Power(BinaryOp):
    def eval(self):
      global result
      result = self.left.eval() 
      for x in range (self.right.eval()-1):
        result = result * self.left.eval()
      return result

    def print(self):
        print("Valor Inicial: ", self.left.eval(),"\nGrau de potencia:  ", self.right.eval(),"\nValor após pontencialização: ", result)

class Attrib(BaseBox):
    def __init__(self, id, expression):
        self.id = id
        self.expression = expression
    def print(self):
        return print("Nome da variavel: ", self.id, "\nExpressão restante: ", self.expression)

class Factor(BaseBox):
  def __init__(self, number):
    self.number = number

class TypeVar(BaseBox):
    def __init__(self, typeVar):
        self.typeVar = typeVar

    def print(self):
        return print(self.typeVar)

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV','ID','EQUALS', 'POWER'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV']), 
        ('left', ['POWER'])
    ]
)

# regra <atrib>::= ID "=" <expression>

@pg.production('atrib : ID EQUALS expression')
def attrib(p):
  return Attrib(p[0].getstr(),p[2])

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression POWER expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    elif p[1].gettokentype() == 'POWER':
        return Power(left, right)
      
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

#Testes

In [ ]:
arvore=parser.parse(lexer.lex('x=2+4'))
print(arvore)
print(arvore.id)
print(arvore.print())

print(arvore.expression.eval())
print(arvore.expression.print())